In [25]:
import cv2
import os
import numpy as np

def extract_frames(video_path, output_folder, frame_interval=30, resize_factor=1):
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = int(fps * frame_interval)  # Adjust frame interval based on FPS
    
    print(f"FPS: {fps}")
    print(f"Total Frames: {total_frames}")
    print(f"Frame Interval: {frame_interval}")
    
    frame_count = 0
    saved_frame_count = 0
    frames_array = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        if frame_count % frame_interval == 0:
            # Resize the frame
            height, width = frame.shape[:2]
            new_dimensions = (int(width / resize_factor), int(height / resize_factor))
            resized_frame = cv2.resize(frame, new_dimensions, interpolation=cv2.INTER_AREA)
            
            # Save the resized frame
            frame_filename = os.path.join(output_folder, f"frame_{saved_frame_count:04d}.jpg")
            cv2.imwrite(frame_filename, resized_frame)
            print(f"Saved: {frame_filename}")
            
            # Append the resized frame to the frames array
            frames_array.append(resized_frame)
            
            saved_frame_count += 1
        
        frame_count += 1
    
    cap.release()
    print("Frame extraction completed.")
    
    return np.array(frames_array)

# Example usage
video_path = '../demoVideos/1-Zenmuse_X4S_Clip2.mp4'
# video_path = "C:/Users/vikas/Downloads/1-Zenmuse_X4S_1.mp4"
output_folder = '../images/'
frame_interval_seconds = 2  # Save every 1 second
resize_factor = 4 # Decrease image size by a factor of 2
frames_array = extract_frames(video_path, output_folder, frame_interval=frame_interval_seconds, resize_factor=resize_factor)

print(f"Extracted {len(frames_array)} frames.")


FPS: 30.0
Total Frames: 1800
Frame Interval: 60
Saved: ../images/frame_0000.jpg
Saved: ../images/frame_0001.jpg
Saved: ../images/frame_0002.jpg
Saved: ../images/frame_0003.jpg
Saved: ../images/frame_0004.jpg
Saved: ../images/frame_0005.jpg
Saved: ../images/frame_0006.jpg
Saved: ../images/frame_0007.jpg
Saved: ../images/frame_0008.jpg
Saved: ../images/frame_0009.jpg
Saved: ../images/frame_0010.jpg
Saved: ../images/frame_0011.jpg
Saved: ../images/frame_0012.jpg
Saved: ../images/frame_0013.jpg
Saved: ../images/frame_0014.jpg
Saved: ../images/frame_0015.jpg
Saved: ../images/frame_0016.jpg
Saved: ../images/frame_0017.jpg
Saved: ../images/frame_0018.jpg
Saved: ../images/frame_0019.jpg
Saved: ../images/frame_0020.jpg
Saved: ../images/frame_0021.jpg
Saved: ../images/frame_0022.jpg
Saved: ../images/frame_0023.jpg
Saved: ../images/frame_0024.jpg
Saved: ../images/frame_0025.jpg
Saved: ../images/frame_0026.jpg
Saved: ../images/frame_0027.jpg
Saved: ../images/frame_0028.jpg
Saved: ../images/frame_0

In [26]:
imageStitcher = cv2.Stitcher_create()

error, stitched_img = imageStitcher.stitch(frames_array)

cv2.imwrite('../results/output.png', stitched_img)
cv2.imshow('Stitched Image', stitched_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
if not error:
    img_border = cv2.copyMakeBorder(stitched_img, 10,10,10,10, cv2.BORDER_CONSTANT, (0,0,0))
    
    gray = cv2.cvtColor(img_border, cv2.COLOR_BGR2GRAY)
    thresh_img = cv2.threshold(gray, 0 ,255, cv2.THRESH_BINARY)[1]
    
    cv2.imshow('Threshold Image', thresh_img)
    cv2.waitKey(0)
    
    contours, hier = cv2.findContours(thresh_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    areaOI = max(contours, key=cv2.contourArea) 
    
    mask = np.zeros(thresh_img.shape, dtype='uint8')
    x, y, w, h = cv2.boundingRect(areaOI)
    cv2.rectangle(mask, (x,y), (x+w, y+h), 255, -1)
    
    minRectangle = mask.copy()
    sub = mask.copy()
    
    while cv2.countNonZero(sub) > 0:
        minRectangle = cv2.erode(minRectangle, None)
        sub = cv2.subtract(minRectangle, thresh_img)
        
    contours, hier = cv2.findContours(minRectangle.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    areaOI = max(contours, key=cv2.contourArea)
    
    cv2.imshow('minRectImage', minRectangle)
    cv2.waitKey(0)
    
    x, y, w, h = cv2.boundingRect(areaOI)
    
    processed = stitched_img[y:y+h, x:x+w]
    
    cv2.imwrite('../results/processed.png', processed)
    cv2.imshow('Stitched Image', processed)
    cv2.waitKey(0)
    cv2.destroyAllWindows()